In [1]:
import pandas as pd
import numpy as np

# from bokeh.io import output_file
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
output_notebook()

Loading BokehJS ...

In [2]:
#get data
original_Enel_1min = pd.read_csv(r"C:\Users\Caleb\Documents\GitHub\CPSproject\app_development\data\combined_Enel_energy_balance_within_10W_1min.csv")

In [3]:
#index by time
original_Enel_1min["time"]= pd.to_datetime(original_Enel_1min["time"])
original_Enel_1min.set_index('time', inplace=True)

In [4]:

new_df=original_Enel_1min.drop(columns= ["Unnamed: 0","SoC","Battery"])

In [5]:
grouped_og=original_Enel_1min.groupby("dataid").count()
grouped_og

,Unnamed: 0,state,car1,grid,Load_+_Battery(Charging),PV_+_Battery(Discharge),Battery,SoC
dataid,,,,,,,,
500,305369,305369,305369,305369,305369,305369,305369,305369
501,260414,260414,260414,260414,260414,260414,260414,260414
502,337770,337770,337770,337770,337770,337770,337770,337770
503,358570,358570,358570,358570,358570,358570,358570,358570
504,252752,252752,252752,252752,252752,252752,252752,252752
...,...,...,...,...,...,...,...,...
604,147621,147621,147621,147621,147621,147621,147621,147621
605,78148,78148,78148,78148,78148,78148,78148,78148
606,139876,139876,139876,139876,139876,139876,139876,139876


In [6]:
dataids= grouped_og.index.values
dataids
# dataids[55:len(dataids)]

array([500, 501, 502, 503, 504, 505, 506, 507, 512, 513, 514, 515, 516,
       517, 518, 519, 520, 522, 523, 525, 526, 527, 529, 533, 535, 536,
       537, 539, 540, 541, 542, 544, 546, 547, 548, 549, 551, 554, 555,
       557, 559, 560, 561, 563, 564, 565, 566, 568, 569, 570, 571, 572,
       573, 575, 576, 578, 579, 580, 582, 583, 585, 586, 587, 590, 591,
       592, 593, 594, 596, 597, 598, 600, 602, 603, 604, 605, 606, 607,
       609], dtype=int64)

In [7]:
new_df["state"][-1]

'Brescia'

In [8]:

compute_dataids = dataids #specify the houses you want to convert to 15 min data

#Resample
for dataid in compute_dataids:
    #start with this first section of df
    if dataid == compute_dataids[0]:
        df_iter= new_df.loc[new_df["dataid"]==dataid,:]
        state=df_iter["state"][0]
        df_iter = df_iter.resample('15T').mean()
        df_iter["state"]= state
        df_cum=df_iter
    else:
        df_iter= new_df.loc[new_df["dataid"]==dataid,:] 
        state=df_iter["state"][0] #grab state name
        df_iter= df_iter.resample('15T').mean() 
        df_iter["state"]= state
        df_cum = df_cum.append(df_iter, ignore_index=False)
        
        
state_resampled_15min = df_cum

In [9]:
state_resampled_15min.groupby("dataid").count()
state_resampled_15min

,dataid,car1,grid,Load_+_Battery(Charging),PV_+_Battery(Discharge),state
time,,,,,,
2020-01-01 00:00:00,500.0,0.0,19.600000,327.666667,307.466667,Modena
2020-01-01 00:15:00,500.0,0.0,7.266667,556.933333,549.133333,Modena
2020-01-01 00:30:00,500.0,0.0,-6.076923,593.153846,599.153846,Modena
2020-01-01 00:45:00,500.0,0.0,4.933333,576.666667,571.066667,Modena
2020-01-01 01:00:00,500.0,0.0,-16.733333,446.466667,463.400000,Modena
...,...,...,...,...,...,...
2020-09-30 18:30:00,609.0,0.0,23.066667,1084.266667,1061.400000,Brescia
2020-09-30 18:45:00,609.0,0.0,16.000000,572.400000,556.666667,Brescia
2020-09-30 19:00:00,609.0,0.0,22.466667,562.933333,540.400000,Brescia


In [10]:
#write our new data to csv's
state_resampled_15min.to_csv(r"C:\Users\Caleb\Documents\GitHub\CPSproject\app_development\data\combined_Enel_with_Battery_15min_resamp_energy_balanced_10W.csv")


In [14]:
state_resampled_15min.loc[state_resampled_15min["Load_+_Battery(Charging)"].isnull(),:]

,dataid,car1,grid,Load_+_Battery(Charging),PV_+_Battery(Discharge),state
time,,,,,,
2020-01-19 00:00:00,NaN,NaN,NaN,NaN,NaN,Modena
2020-01-19 05:00:00,NaN,NaN,NaN,NaN,NaN,Modena
2020-01-27 14:45:00,NaN,NaN,NaN,NaN,NaN,Modena
2020-01-27 15:00:00,NaN,NaN,NaN,NaN,NaN,Modena
2020-01-27 15:15:00,NaN,NaN,NaN,NaN,NaN,Modena
...,...,...,...,...,...,...
2020-09-29 20:00:00,NaN,NaN,NaN,NaN,NaN,Brescia
2020-09-29 20:15:00,NaN,NaN,NaN,NaN,NaN,Brescia
2020-09-29 20:30:00,NaN,NaN,NaN,NaN,NaN,Brescia
